In [478]:
import numpy as np
import yfinance as yf

In [479]:
df = yf.download('TCS.NS', start='2023-01-01', end='2023-12-31', interval='1d', progress=False, multi_level_index=False, auto_adjust=False)

In [480]:
def MACD(df):
  df['Short EMA'] = df['Close'].ewm(span=9, adjust=False).mean()
  df['Long EMA'] = df['Close'].ewm(span=12, adjust=False).mean()
  df['MACD'] = df['Short EMA'] - df['Long EMA']
  df['Signal Line'] = df['MACD'].ewm(span=9, adjust=False).mean()
  return df

In [481]:
def RSI(df):
  df['Difference'] = df['Close'].diff()
  df['Gains'] = np.where(df['Difference'] > 0, df['Difference'], 0)
  df['Avg Gains'] = df['Gains'].rolling(window=14).mean()
  df['Loss'] = abs(np.where(df['Difference'] < 0, df['Difference'], 0))
  df['Avg Loss'] = df['Loss'].rolling(window=14).mean()
  df['RS'] = df['Avg Gains'] / df['Avg Loss']
  df['RSI'] = (1 - (1 / (1 + df['RS']))) * 100
  return df

In [482]:
def BB(df):
  df['Middle Band'] = df['Close'].rolling(window=20, min_periods=1).mean()
  df['20 STD'] = df['Close'].rolling(window=20, min_periods=1).std()
  df['Upper Band'] = df['Middle Band'] + (2 * df['20 STD'])
  df['Lower Band'] = df['Middle Band'] - (2 * df['20 STD'])
  return df

In [483]:
def ATR(df):
  df['High Low Difference'] = df['High'] - df['Low']
  df['Prev High Low Difference'] = df['High'].shift(periods=1) - df['Low']
  df['High Prev Low Difference'] = df['High'] - df['Low'].shift(periods=1)
  df['True Range'] = np.abs(df[['High Low Difference', 'Prev High Low Difference', 'High Prev Low Difference']].max(axis=1))
  df['ATR'] = df['True Range'].ewm(alpha=1/14, adjust=False).mean()
  return df

In [484]:
def generate_signal(df):
  macd_buy_signal = df['MACD'] > df['Signal Line']
  rsi_buy_signal = (df['RSI'] > 30) & (df['RSI'] < 70)
  bollinger_buy_signal = df['Close'] > df['Lower Band']
  buy_signal = macd_buy_signal & rsi_buy_signal & bollinger_buy_signal

  macd_sell_signal = df['MACD'] < df['Signal Line']
  rsi_sell_signal = df['RSI'] > 70
  bollinger_sell_signal = df['Close'] > df['Upper Band']
  sell_signal = macd_sell_signal | rsi_sell_signal | bollinger_sell_signal

  df['Signal'] = 0
  df.loc[buy_signal, 'Signal'] = 1
  df.loc[sell_signal, 'Signal'] = -1
  return df

In [485]:
def run_backtest(df, stop_loss_percent=1, take_profit_percent=4, initial_capital=1_00_000):
    capital = initial_capital
    position = 0
    entry_price = 0
    trades = []
    equity_curve = []

    for i in range(len(df)):
        price = df.iloc[i]['Close']
        signal = df.iloc[i]['Signal']

        if position == 0 and signal == 1:
            qty = capital // price
            if qty > 0:
              position = qty
              entry_price = price
              capital -= qty * price

        if position > 0:
            stop_loss_price = entry_price * (1 - stop_loss_percent / 100) # Default stop loss percentage is 1
            take_profit_price = entry_price * (1 + take_profit_percent / 100) # Default take profit percentage is 4

            if price <= stop_loss_price or price >= take_profit_price:
                pnl = position * (price - entry_price)
                capital += position * price
                trades.append(pnl)
                position = 0

        if position > 0 and signal == -1:
            pnl = position * (price - entry_price)
            capital += position * price
            trades.append(pnl)
            position = 0

        # Equity tracking
        equity = capital
        if position > 0:
            equity += position * price
        equity_curve.append(equity)

    # Force square-off
    final_price = df.iloc[-1]["Close"]
    if position > 0:
        pnl = position * (final_price - entry_price)
        capital += position * final_price
        trades.append(pnl)

    # Metrics
    trades = np.array(trades)
    wins = trades[trades > 0]
    total_pnl = capital - initial_capital
    return_pct = (total_pnl / initial_capital) * 100

    equity_curve = np.array(equity_curve)
    returns = np.diff(equity_curve) / equity_curve[:-1]
    sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(len(returns))
    drawdown = equity_curve / np.maximum.accumulate(equity_curve) - 1
    max_drawdown = drawdown.min() * 100

    print(f"PnL: {round(total_pnl, 2)}")
    print(f"Sharpe Ratio: {round(sharpe_ratio, 2)}")
    print(f"Max Drawdown (%): {round(max_drawdown, 2)}")
    print(f"Number of Trades: {len(trades)}")
    print(f"Number of Winning Trades: {len(wins)}")
    print(f"Number of Losing Trades: {len(trades) - len(wins)}")

In [486]:
df = RSI(df)
df = MACD(df)
df = BB(df)
df = ATR(df)

df = generate_signal(df)
run_backtest(df)

PnL: 13110.65
Sharpe Ratio: 1.15
Max Drawdown (%): -5.96
Number of Trades: 24
Number of Winning Trades: 14
Number of Losing Trades: 10
